Bert(Bidirectional Encoder Representation from Transformers) 是个只有**双向编码器**的 Transformer，虽然结构很简单，但效果很好
分为两个版本：
- Base: #blocks 12, hidden size 768,    #heads 12,  #parameters 110M
- Large:#blocks 24, hidden size 1024,   #heads 16,  #parameters 340M

创新点：
- 输入
- Loss 函数

<cls> 表示句子的分类，<sep> 表示两句话之间的断句符号

### 训练任务1：带掩码的语言模型
Bert 之所以是双向的，是因为它不需要预测未来，只做完形填空。
带掩码的语言模型每次随机（15%概率)将一些词元换成\<mask>
因为微调任务中不出现\<mask>
- 80%概率下，将选中的词元变成\<mask>
- 10%概率下换成一个随机词元
- 10%概率下保持原有的词元

目的是不要看到 mask 就预测

### 训练任务2：下一句子预测
- 预测一个句子对中两个句子是不是相邻
>每次给出一对句子，预测句子在原始数据中是不是相邻的。

- 因此，在训练样本中：
    - 50% 概率选择**相邻**句子对：<cls>this movie is great<Sep>i like it <sep>
    - 50% 概率选择**随机**句子对：<cls>this movie is great<Sep>hello world<sep>

### 总结
- BERT针对微调设计
- 基于Transformer的编码器做了如下修改
    - 模型更大，训川练数据更多
    - 输入句子对，片段嵌入，可学习的位置编码
- 训练时使用两个任务：
    - 带掩码的语言模型
    - 下一个句子预测

## Bert 模型

In [2]:
import torch
from torch import nn
from d2l import torch as d2l

### 1. 输入表示
NLP 有些任务（如情感分析）以单个文本作为输入，而有些任务（如自然语言推断）以一对文本序列作为输入。bert 对此进行了明确的区分。
- 输入为单文本时，输入序列为 \<cls> + 文本序列(标记化表示)$e_A$ + \<sep>。
- 输入为文本对时，输入序列为 \<cls> + 第一个文本序列的标记$e_A$ + \<sep> + 第二个文本序列标记$e_B$ + \<sep>

In [3]:
def get_tokens_and_segments(tokens_a, tokens_b=None):
    """获取输入序列的词元及其片段索引"""
    tokens = ['<cls>'] + tokens_a + ['<sep>']
    # 0和1分别标记片段A和B
    segments = [0] * (len(tokens_a) + 2)
    if tokens_b is not None:
        tokens += tokens_b + ['<sep>']
        segments += [1] * (len(tokens_b) + 1)
    # 返回的是一个单词数组，及等大的段嵌入标记数组(标记当前词元属于 e_A 还是 e_B)
    return tokens, segments

BERT输入序列的嵌入是**词元嵌入**、**片段嵌入**和**位置嵌入**的和。

BERTEncoder 类类似于 TransformerEncoder 类。
但不同在于，BERTEncoder 使用片段嵌入和可学习的位置嵌入。「可学习的位置嵌入」意思是变量应该被nn.Parameter() 包裹

In [4]:
class BERTEncoder(nn.Module):
    """BERT编码器"""
    def __init__(self, vocab_size, num_hiddens, norm_shape, ffn_num_input,ffn_num_hiddens,
                 num_heads, num_layers, dropout,
                 max_len=1000, key_size=768, query_size=768, value_size=768,
                 **kwargs):
        super(BERTEncoder, self).__init__(**kwargs)
        self.token_embedding = nn.Embedding(vocab_size, num_hiddens)

        #  BERT 任务只分单句(用 0 表示)和句子对(两句分别用 0 和 1 表示)，因此num_embeddings == 2
        self.segment_embedding = nn.Embedding(2, num_hiddens)

        self.blks = nn.Sequential()
        for i in range(num_layers):
            self.blks.add_module(f"{i}", d2l.EncoderBlock(
                key_size, query_size, value_size, num_hiddens, norm_shape,
                ffn_num_input, ffn_num_hiddens, num_heads, dropout, True))
        # 在BERT中，位置嵌入是可学习的，因此我们创建一个足够长的位置嵌入参数
        self.pos_embedding = nn.Parameter(torch.randn(1, max_len,num_hiddens))

    def forward(self, tokens, segments, valid_lens):
        # 在以下代码段中，X的形状保持不变：（批量大小，最大序列长度，num_hiddens）
        X = self.token_embedding(tokens) + self.segment_embedding(segments)
        X = X + self.pos_embedding.data[:, :X.shape[1], :]
        # 数据要在每一层间传递更新
        for blk in self.blks:
            X = blk(X, valid_lens)
        return X

设词表大小为10000，为了演示 BERTEncoder 的前向推断，让我们创建一个实例并初始化它的参数。

In [5]:
vocab_size, num_hiddens, ffn_num_hiddens, num_heads = 10000, 768, 1024, 4
norm_shape, ffn_num_input, num_layers, dropout = [768], 768, 2, 0.2
encoder = BERTEncoder(vocab_size, num_hiddens, norm_shape, ffn_num_input,
                      ffn_num_hiddens, num_heads, num_layers, dropout)

In [6]:
tokens = torch.randint(0, vocab_size, (2, 8)) # 表示一批数据，包括 2 个长度均为 8 个单词的句子，每个单词的 BERT 输出是一个 768 维(构造编码器时的num_hiddens 参数)的向量
segments = torch.tensor([[0, 0, 0, 0, 1, 1, 1, 1], [0, 0, 0, 1, 1, 1, 1, 1]])

encoded_X = encoder(tokens, segments, None)
encoded_X.shape

torch.Size([2, 8, 768])

↑ batch_size = 2;最长句的长度max_len = 8;每个单词的 BERT 表示=768

### 2. 预训练任务
BERTEncoder的前向推断给出了输入文本的每个词元和插入的特殊标记`"<cls>"`及`"<seq>"`的BERT表示。
接下来，我们将使用这些表示来计算预训练 BERT 的损失函数。预训练包括以下两个任务：掩蔽语言模型和下一句预测。

#### (1) 任务一：遮蔽语言模型 Masked Language Modeling
为了双向编码上下文以表示每个词元，BERT (1)随机掩蔽词元(2)并使用来自双向上下文的词元以自监督的方式**预测**掩蔽词元。此任务称为掩蔽语言模型。

**具体操作**：
随机遮蔽 15% 的词元，作为预测的掩蔽词元，替换规则为：
- 80% 时间用 `"<mask>"` 替换(例如，“this movie is great”变为“this movie is \<mask>”)
- 10% 时间为随机词元(例如，“this movie is great”变为“this movie is drink”)
- 10% 时间内不变(例如，“this movie is great”变为“this movie is great”)

设置 10% 的随机词元的好处在于，BERT在其双向上下文编码中不那么偏向于掩蔽词元（尤其是当标签词元保持不变时）。

下面的 MaskLM 类来预测 BERT 预训练的掩蔽语言模型任务中的掩蔽标记。
预测使用单隐藏层的多层感知机（self.mlp）。在前向推断中，它需要两个输入：
1. BERTEncoder的编码结果
2. 用于预测的词元位置。
输出是这些位置的预测结果。

In [7]:
#@save
class MaskLM(nn.Module):
    """BERT的掩蔽语言模型任务"""
    def __init__(self, vocab_size, num_hiddens, num_inputs=768, **kwargs):
        super(MaskLM, self).__init__(**kwargs)
        # 这个单隐藏层的 MLP 接受的是BERT的输出 (batch_size,max_len,BERTEncoder.num_hiddens=768)
        # 因此 mlp.num_input = BERTEncoder.num_hiddens = 768
        self.mlp = nn.Sequential(nn.Linear(num_inputs, num_hiddens),
                                 nn.ReLU(),
                                 nn.LayerNorm(num_hiddens),
                                 nn.Linear(num_hiddens, vocab_size)) # 最终是要识别是哪个单词，输出层=vocab_size

    # X                 是刚刚 BERT 的输出        (batch_size,max_len,num_hidden=768)
    # pred_positions    用来指示被遮蔽的词的位置    (batch_size,max_len)
    def forward(self, X, pred_positions):
        # 获取 max_len ,即最长句子的单词数/列数
        num_pred_positions = pred_positions.shape[1]

        # reshape(-1)表示将(m,n,k)多维张量转换为长为 (m*n*k,) 的一维张量。
        pred_positions = pred_positions.reshape(-1)

        # 举例假设batch_size=2，num_pred_positions=3。
        batch_size = X.shape[0]
        batch_idx = torch.arange(0, batch_size) # batch_idx = np.array([0,1])
        batch_idx = torch.repeat_interleave(batch_idx, num_pred_positions) # batch_idx = np.array([0,0,0,1,1,1])

        # 假设 pred_positions = [[1, 5, 2], [6, 1, 5]]
        # masked_X = [X[0,1],X[0,5],X[0,2],X[1,6],X[1,1],X[1,5]]  masked_X.shape = (6,768)
        masked_X = X[batch_idx, pred_positions]
        masked_X = masked_X.reshape((batch_size, num_pred_positions, -1)) # masked_X.shape = (2,3,768)
        mlm_Y_hat = self.mlp(masked_X)
        # masked_Y.shape = (2,3,vocab_size)
        return mlm_Y_hat

演示1-2 MaskLM 中 forward() 内 masked_X 的实质是选择 batch 中的每句话中的被遮蔽单词

In [13]:
batch_idx = torch.tensor([0,0,0,1,1,1])
mlm_positions = torch.tensor([1, 5, 2,6, 1, 5])
masked_X = encoded_X[batch_idx, mlm_positions]
masked_X.shape

torch.Size([6, 768])

演示2-2 MaskLM 的forward()
来自 BERTEncoder 的正向推断 encoded_X 表示 2 个 BERT 输入序列。
我们将 mlm_positions 定义为在 encoded_X 的任一输入序列中预测的3个指示(即 batch 中每句话都要求遮蔽 3 个元素)。
mlm 的前向推断返回encoded_X 的所有掩蔽位置 mlm_positions 处的预测结果 mlm_Y_hat 。
每个预测 = vocab_size。

In [14]:
mlm = MaskLM(vocab_size, num_hiddens)
mlm_positions = torch.tensor([[1, 5, 2], [6, 1, 5]])
mlm_Y_hat = mlm(encoded_X, mlm_positions)
mlm_Y_hat.shape

torch.Size([2, 3, 10000])

通过掩码下的预测词元mlm_Y的真实标签mlm_Y_hat，我们可以计算在BERT预训练中的遮蔽语言模型任务的交叉熵损失。

In [9]:
mlm_Y = torch.tensor([[7, 8, 9], [10, 20, 30]])
loss = nn.CrossEntropyLoss(reduction='none')
mlm_l = loss(mlm_Y_hat.reshape((-1, vocab_size)), mlm_Y.reshape(-1))
mlm_l.shape

torch.Size([6])

#### (2) 任务二：下一句预测
尽管掩蔽语言建模能够编码双向上下文来表示单词，但它不能显式地**建模文本对之间的逻辑关系**。
为了帮助理解两个文本序列之间的关系，设置下一句预测类 NextSentencePred 。

在为预训练生成句子对时，有一半的时间它们确实是标签为“真”的连续句子；在另一半的时间里，第二个句子是从语料库中随机抽取的，标记为“假”。
NextSentencePred 类使用单隐藏层的多层感知机来预测 *第二个句子是否是BERT输入序列中第一个句子的下一个句子*。

由于Transformer编码器中的自注意力，特殊词元`"<cls>"`的BERT表示已经对输入的两个句子进行了编码。
因此，多层感知机分类器的输出层（self.output）以X作为输入，其中X是多层感知机隐藏层的输出，而MLP隐藏层的输入是编码后的`<cls>`词元。

In [16]:
class NextSentencePred(nn.Module):
    """BERT的下一句预测任务"""
    def __init__(self, num_inputs, **kwargs):
        super(NextSentencePred, self).__init__(**kwargs)
        self.output = nn.Linear(num_inputs, 2) # 只做真/假判断

    def forward(self, X):
        # X的形状：(batchsize,num_hiddens)
        return self.output(X)

返回每个 BERT 输入序列的二分类预测

In [17]:
encoded_X = torch.flatten(encoded_X, start_dim=1)
# NSP的输入形状:(batchsize，num_hiddens)
nsp = NextSentencePred(encoded_X.shape[-1])
nsp_Y_hat = nsp(encoded_X)
nsp_Y_hat.shape

torch.Size([2, 2])

In [18]:
# 计算两个二元分类的交叉熵损失。
nsp_y = torch.tensor([0, 1])
nsp_l = loss(nsp_Y_hat, nsp_y)
nsp_l.shape

torch.Size([2])

### 3. 整合代码
在预训练BERT时，最终的损失函数是掩蔽语言模型损失函数和下一句预测损失函数的线性组合。
froward() 返回
- 编码后的 BERT 表示 `encoded_X`
- 掩蔽语言模型预测 `mlm_Y_hat`
- 下一句预测 `nsp_Y_hat`

In [19]:
class BERTModel(nn.Module):
    """BERT模型"""
    def __init__(self, vocab_size, num_hiddens, norm_shape, ffn_num_input,
                 ffn_num_hiddens, num_heads, num_layers, dropout,
                 max_len=1000, key_size=768, query_size=768, value_size=768,
                 hid_in_features=768, mlm_in_features=768,nsp_in_features=768):
        super(BERTModel, self).__init__()
        self.encoder = BERTEncoder(vocab_size, num_hiddens, norm_shape,
                    ffn_num_input, ffn_num_hiddens, num_heads, num_layers,
                    dropout, max_len=max_len, key_size=key_size,
                    query_size=query_size, value_size=value_size)
        self.hidden = nn.Sequential(nn.Linear(hid_in_features, num_hiddens),
                                    nn.Tanh())
        self.mlm = MaskLM(vocab_size, num_hiddens, mlm_in_features)
        self.nsp = NextSentencePred(nsp_in_features)

    def forward(self, tokens, segments, valid_lens=None,pred_positions=None):
        encoded_X = self.encoder(tokens, segments, valid_lens)
        if pred_positions is not None:
            mlm_Y_hat = self.mlm(encoded_X, pred_positions)
        else:
            mlm_Y_hat = None
        # 用于下一句预测的多层感知机分类器的隐藏层，0是“<cls>”标记的索引,对应一个长为 num_in_feature 的嵌入向量
        nsp_Y_hat = self.nsp(self.hidden(encoded_X[:, 0, :])) # (batch_size,1,hid_in_features) -> (batch_size,1,num_hiddens) -> (batch_size,1,2)
        return encoded_X, mlm_Y_hat, nsp_Y_hat

## Bert 数据

## Bert 训练

## QA